# Gestion de Pedidos de una Cafeteria

In [36]:
class Persona:

    lista=[]

    def __init__(self,nombre,correo):
        self.nombre = nombre
        self.correo = correo
    
    def registrar(self):
        Persona.lista.append(self)
        print(f"La persona {self.nombre} fue registrada con el correo {self.correo}")

    def actualizar_datos(self,nombre,correo):
        self.nombre=nombre
        self.correo=correo
        print(f"Los datos han sido actualizados")

    @classmethod
    def personas_registradas(cls):
        print("Personas registradas")
        for Persona in cls.lista:
            print(f"-{Persona.nombre} - {Persona.correo}")       

In [37]:
class Cliente(Persona):
    def __init__(self,nombre,correo):
        super().__init__(nombre,correo)
        self.consultar_historial = []

    def hacer_pedido(self,producto,cantidad, inventario):
        if inventario.checar_stock(producto, cantidad):
            inventario.actualizar_stock(producto, cantidad)
            self.consultar_historial.append({"producto": producto, "cantidad": cantidad})
            print(f"Pedido realizado para el producto: {producto.nombre}, cantidad: {cantidad}")
        else:
            print("No hay suficientes productos disponibles.")

In [38]:
class Empleado(Persona):
    def __init__(self, nombre, correo, rol):
        super().__init__(nombre, correo)
        self.rol = rol

    def agregar_comida(self, platillo, inventario):
        if inventario.chequear_stock(platillo, 1):
            inventario.actualizar_stock(platillo, 1)
            print(f"Comida añadida: {platillo.nombre}")
        else:
            print(f"No hay suficientes productos para agregar {platillo.nombre}.")

    def agregar_bebida(self, bebida, inventario):
        if inventario.chequear_stock(bebida, 1):
            inventario.actualizar_stock(bebida, 1)
            print(f"Bebida agregada: {bebida.nombre}")
        else:
            print(f"No hay suficientes productos para agregar {bebida.nombre}.")

In [39]:
class Producto_Base:
    def __init__(self,nombre,precio):
        self.nombre = nombre
        self.precio = precio

In [40]:
class Bebida(Producto_Base):
    def __init__(self,nombre,precio,tamaño,tipo,personalizado=None):
        super().__init__(nombre,precio)
        self.tamaño = tamaño
        self.tipo = tipo
        self.personalizado = personalizado if personalizado else []

    def agregar_ingrediente(self, ingrediente):
        self.personalizado.append(ingrediente)
        print(f"Ingrediente '{ingrediente}' agregado a la bebida {self.nombre}")

    def mostrar_informacion(self):
        ingredientes = ", ".join(self.personalizado) if self.personalizado else "Ninguno"
        print(f"{self.nombre} ({self.tamaño}, {self.tipo}) - Ingredientes adicionales: {ingredientes}")

In [41]:
class Postre(Producto_Base):
    def __init__(self,nombre,precio,es_vegano,sin_gluten):
        super().__init__(nombre,precio)
        self.es_vegano = es_vegano
        self.sin_gluten = sin_gluten

    def mostrar_informacion(self):
        vegano = "Sí" if self.es_vegano else "No"
        gluten = "Sí" if self.sin_gluten else "No"
        print(f"{self.nombre}: Vegano: {vegano}, Sin gluten: {gluten}")

In [42]:
class Inventario:
    def __init__(self):
        self.stock = {}
    def agregar_producto(self, producto, cantidad):
        if producto.nombre in self.stock:
            self.stock[producto.nombre] += cantidad
        else:
            self.stock[producto.nombre] = cantidad

    def chequear_stock(self, producto, cantidad):
        if producto.nombre in self.stock and self.stock[producto.nombre] >= cantidad:
            return True
        return False

    def actualizar_stock(self, producto, cantidad):
        if self.chequear_stock(producto, cantidad):
            self.stock[producto.nombre] -= cantidad
        else:
            print("No hay suficiente stock para actualizar.")

In [43]:
class Pedido:
    def __init__(self):
        self.productos = []
        self.estado = "Pendiente"
        self.total = 0

    def agregar_producto(self, producto, cantidad):
        self.productos.append({"producto": producto, "cantidad": cantidad})
        self.total += producto.precio * cantidad

    def mostrar_pedido(self):
        print(f"Estado del pedido: {self.estado}")
        for item in self.productos:
            print(f"Producto: {item['producto'].nombre}, Cantidad: {item['cantidad']}")
        print(f"Total: ${self.total}")

In [44]:
class Promocion:
    def __init__(self, descuento, condiciones):
        self.descuento = descuento
        self.condiciones = condiciones

    def aplicar_descuento(self, pedido):
        if self.condiciones(pedido):
            descuento = (pedido.total * self.descuento) / 100
            pedido.total -= descuento
            print(f"Descuento aplicado: ${descuento}")
        else:
            print("No se cumple con las condiciones para aplicar el descuento.")     

In [45]:
# Crear productos
bebida1 = Bebida("Cafe", 3, "Pequeño", "Frio")
bebida2 = Bebida("Jugo de betabel", 5, "Grande", "Frio")
postre1 = Postre("Waffles con miel y zarzamora", 7, False, True)
postre2 = Postre("Helado Napolitano", 12, False, True)

# Mostrar info de los productos
postre1.mostrar_informacion()
postre2.mostrar_informacion()
bebida1.mostrar_informacion()

# Agregar ingredientes a las bebidas
bebida1.agregar_ingrediente("Leche de almendra")
bebida1.agregar_ingrediente("Canela")
bebida1.mostrar_informacion()

# Crear inventario
inventario = Inventario()
inventario.agregar_producto(bebida1, 15)
inventario.agregar_producto(bebida2, 9)
inventario.agregar_producto(postre1, 66)
inventario.agregar_producto(postre2, 76)

# Cliente realiza un pedido
cliente1 = Cliente("Gerard", "gerard@mail.com")
cliente1.registrar()

pedido1 = Pedido()
pedido1.agregar_producto(bebida1, 5)
pedido1.agregar_producto(postre2, 4)
pedido1.mostrar_pedido()

# Crear promoción
def condiciones_descuento(pedido):
    return pedido.total >= 30

promocion1 = Promocion(19, condiciones_descuento)
promocion1.aplicar_descuento(pedido1)

pedido1.mostrar_pedido()

# Empleado agrega comida al inventario
empleado1 = Empleado("Minerva", "minervamqvlc@mail.com", "Cajera")
empleado1.agregar_comida(postre2, inventario)

# Mostrar estado del inventario
print("\nEstado del inventario:")
for producto, cantidad in inventario.stock.items():
    print(f"{producto}: {cantidad} piezas")

Waffles con miel y zarzamora: Vegano: No, Sin gluten: Sí
Helado Napolitano: Vegano: No, Sin gluten: Sí
Cafe (Pequeño, Frio) - Ingredientes adicionales: Ninguno
Ingrediente 'Leche de almendra' agregado a la bebida Cafe
Ingrediente 'Canela' agregado a la bebida Cafe
Cafe (Pequeño, Frio) - Ingredientes adicionales: Leche de almendra, Canela
La persona Gerard fue registrada con el correo gerard@mail.com
Estado del pedido: Pendiente
Producto: Cafe, Cantidad: 5
Producto: Helado Napolitano, Cantidad: 4
Total: $63
Descuento aplicado: $11.97
Estado del pedido: Pendiente
Producto: Cafe, Cantidad: 5
Producto: Helado Napolitano, Cantidad: 4
Total: $51.03
Comida añadida: Helado Napolitano

Estado del inventario:
Cafe: 15 piezas
Jugo de betabel: 9 piezas
Waffles con miel y zarzamora: 66 piezas
Helado Napolitano: 75 piezas
